In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

In [3]:
from tensorflow.keras import layers ,activations,models,preprocessing,utils

In [4]:
data="chatbot dataset.txt"

Creating the data for the input_put and target 

In [5]:
input_texts=[]
target_texts=[]
with open(data) as f:
    lines=f.read().split('\n')
for line in lines[:min(600,len(lines) -1)]:
    input_text=line.split('\t')[0]
    target_text=line.split('\t')[1]
    input_texts.append(input_text)
    target_texts.append(target_text)

In [6]:
print("Type of input text :",type(input_text))
print("Type of target_text:",type(target_texts))

Type of input text : <class 'str'>
Type of target_text: <class 'list'>


converting the list in pandas dataframe since input_text,target_text are both are type of list

In [7]:
zippedlist=list(zip(input_texts,target_texts))
lines=pd.DataFrame(zippedlist,columns=['input','output'])


In [8]:
lines.head()

,input,output
0,What are your interests,I am interested in all kinds of things. We can...
1,What are your favorite subjects,"My favorite subjects include robotics, compute..."
2,What are your interests,"I am interested in a wide variety of topics, a..."
3,What is your number,I don't have any number
4,What is your number,23 skiddoo!


Prepareing the data for the encoder


In [9]:
input_line=list()
for line in lines.input:
    input_line.append(line)

In [10]:
print("Input lines are :",input_line)

Input lines are : ['What are your interests', 'What are your favorite subjects', 'What are your interests', 'What is your number', 'What is your number', 'What is your favorite number', 'What can you eat', "Why can't you eat food", 'What is your location', 'What is your location', 'Where are you from', 'Where are you', 'Do you have any brothers', 'Do you have any brothers', 'Who is your father', 'Who is your mother', 'Who is your boss', 'What is your age', 'What is your age', 'what is the illuminati', 'what is the illuminatti', 'what is the illuminatti', 'what is vineland', 'What is Illuminatus', 'What is Illuminatus', 'who wrote vineland', 'who is bilbo baggins', 'who is geoffrey chaucer', 'who is piers anthony', 'have you read plato', 'have you read frankenstein', 'have you ever read a book', 'have you ever read a book', 'have you ever read a book', 'have you read many books', 'have you read homer', 'ray bradbury', 'what is mind children', 'william gibson', 'william gibson', 'holden 

Data preprocessing for encoder

In [11]:
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(input_line)
tokenizer_input=tokenizer.texts_to_sequences(input_line)

lenght_list=list()
for token_seq in tokenizer_input:
    lenght_list.append(len(token_seq))
max_input_length=np.array(lenght_list).max()
print("Input max length is {}".format(max_input_length))
padded_input_lines=preprocessing.sequence.pad_sequences(tokenizer_input,maxlen=max_input_length,padding='post')
encoder_input_data=np.array(padded_input_lines)
print("Encoder input data shape {}".format(encoder_input_data.shape))

input_word_dict=tokenizer.word_index
nump_input_token=len(input_word_dict)+1
print("Number of Input token={}".format(nump_input_token))

Input max length is 22
Encoder input data shape (566, 22)
Number of Input token=518


In [12]:
input_word_dict

{'you': 1,
 'is': 2,
 'are': 3,
 'what': 4,
 'a': 5,
 'do': 6,
 'me': 7,
 'tell': 8,
 'the': 9,
 'joke': 10,
 'not': 11,
 'your': 12,
 'to': 13,
 'how': 14,
 'who': 15,
 'can': 16,
 'of': 17,
 'it': 18,
 'like': 19,
 'i': 20,
 'have': 21,
 'computer': 22,
 'die': 23,
 'eat': 24,
 'stock': 25,
 'about': 26,
 'get': 27,
 'market': 28,
 'hal': 29,
 'ever': 30,
 'in': 31,
 'know': 32,
 'gossip': 33,
 'gossips': 34,
 'robots': 35,
 'will': 36,
 'read': 37,
 'an': 38,
 'when': 39,
 'hi': 40,
 'favorite': 41,
 'robot': 42,
 'up': 43,
 'much': 44,
 'sense': 45,
 'why': 46,
 'should': 47,
 'feel': 48,
 'make': 49,
 'mad': 50,
 'bad': 51,
 'was': 52,
 'immortal': 53,
 'soccer': 54,
 'going': 55,
 'did': 56,
 'does': 57,
 'be': 58,
 'money': 59,
 'guns': 60,
 'making': 61,
 'baseball': 62,
 'play': 63,
 "what's": 64,
 'number': 65,
 'any': 66,
 'wrote': 67,
 'frank': 68,
 'herbert': 69,
 'asimov': 70,
 'anybody': 71,
 'wavelength': 72,
 'far': 73,
 'sad': 74,
 'nice': 75,
 'that': 76,
 'lie': 77,

In [13]:
encoder_input_data

array([[  4,   3,  12, ...,   0,   0,   0],
       [  4,   3,  12, ...,   0,   0,   0],
       [  4,   3,  12, ...,   0,   0,   0],
       ...,
       [498, 499,   9, ...,   0,   0,   0],
       [  9, 207, 208, ...,   0,   0,   0],
       [510, 192,   5, ...,   0,   0,   0]], dtype=int32)

Prepare the input data for the decoder

In [14]:
output_lines=list()
for line in lines.output:
    output_lines.append('<START>'+line+"<END>")

In [15]:
tokenizer=preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(output_lines)
token_output=tokenizer.texts_to_sequences(output_lines)

In [16]:
length_list=list()
for token_seq in token_output:
    length_list.append(len(token_seq))
max_output=np.array(length_list).max()
print("Output length{}".format(max_output))

padded_output=preprocessing.sequence.pad_sequences(token_output,maxlen=max_output,padding='post')
decoder_input_data=np.array(padded_output)
print("Decoder input data shape :{}".format(decoder_input_data.shape))
output_worddic=tokenizer.word_index
num_out_token=len(output_worddic)+1
print("Number of Output tokens= {}".format(num_out_token))

Output length74
Decoder input data shape :(566, 74)
Number of Output tokens= 1692


Preparing the data for the decoder

In [17]:
decoder_target_data=list()
for token_seq in token_output:
    decoder_target_data.append(token_seq[1:])
padded_output_lines=preprocessing.sequence.pad_sequences(decoder_target_data,maxlen=max_output,padding='post')
onehot_output_lines=utils.to_categorical(padded_output_lines,num_out_token)
decoder_target_data=np.array(onehot_output_lines)
print("Decoder target shape {}".format(decoder_target_data.shape))

Decoder target shape (566, 74, 1692)


Defining the model

In [18]:
encoder_inputs=tf.keras.layers.Input(shape=(None,))
encoder_embedding=tf.keras.layers.Embedding(nump_input_token,256,mask_zero=True)(encoder_inputs)
encoder_outputs,state_h,state_c=tf.keras.layers.LSTM(256,return_state=True,recurrent_dropout=0.2,dropout=0.2)(encoder_embedding)
encoder_states=[state_h,state_c]

####################################################

decoder_inputs=tf.keras.layers.Input(shape=(None,))
decoder_embedding=tf.keras.layers.Embedding(num_out_token,256,mask_zero=True)(decoder_inputs)
decoder_lstm=tf.keras.layers.LSTM(256,return_state=True,return_sequences=True,recurrent_dropout=0.2,dropout=0.2)
decoder_outputs,_,_=decoder_lstm(decoder_embedding,initial_state=encoder_states)

decoder_dense=tf.keras.layers.Dense(num_out_token,activation=tf.keras.activations.softmax)
output=decoder_dense(decoder_outputs)

model=tf.keras.models.Model([encoder_inputs,decoder_inputs],output)

model.compile(optimizer=tf.keras.optimizers.Adam(),loss="categorical_crossentropy")

model.summary()


2023-02-27 20:40:37.480147: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-27 20:40:37.480169: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-27 20:40:37.480189: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kali): /proc/driver/nvidia/version does not exist
2023-02-27 20:40:37.480390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 256)    132608      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 256)    433152      ['input_2[0][0]']                
                                                                                              

In [19]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=124,epochs=500)


Epoch 1/500
5/5 [==============================] - 12s 2s/step - loss: 7.4263
Epoch 2/500
5/5 [==============================] - 8s 2s/step - loss: 7.3269
Epoch 3/500
5/5 [==============================] - 8s 2s/step - loss: 6.6426
Epoch 4/500
5/5 [==============================] - 8s 2s/step - loss: 5.9646
Epoch 5/500
5/5 [==============================] - 8s 2s/step - loss: 5.8321
Epoch 6/500
5/5 [==============================] - 8s 2s/step - loss: 5.7437
Epoch 7/500
5/5 [==============================] - 8s 2s/step - loss: 5.7140
Epoch 8/500
5/5 [==============================] - 8s 2s/step - loss: 5.6708
Epoch 9/500
5/5 [==============================] - 8s 2s/step - loss: 5.6525
Epoch 10/500
5/5 [==============================] - 8s 2s/step - loss: 5.6186
Epoch 11/500
5/5 [==============================] - 8s 2s/step - loss: 5.5843
Epoch 12/500
5/5 [==============================] - 8s 2s/step - loss: 5.5436
Epoch 13/500
5/5 [==============================] - 8s 2s/step - loss: 5

5/5 [==============================] - 8s 2s/step - loss: 2.4972
Epoch 107/500
5/5 [==============================] - 8s 2s/step - loss: 2.4720
Epoch 108/500
5/5 [==============================] - 8s 2s/step - loss: 2.4494
Epoch 109/500
5/5 [==============================] - 8s 2s/step - loss: 2.4216
Epoch 110/500
5/5 [==============================] - 8s 2s/step - loss: 2.3953
Epoch 111/500
5/5 [==============================] - 8s 2s/step - loss: 2.3694
Epoch 112/500
5/5 [==============================] - 8s 2s/step - loss: 2.3473
Epoch 113/500
5/5 [==============================] - 8s 1s/step - loss: 2.3208
Epoch 114/500
5/5 [==============================] - 8s 2s/step - loss: 2.2963
Epoch 115/500
5/5 [==============================] - 8s 2s/step - loss: 2.2685
Epoch 116/500
5/5 [==============================] - 8s 2s/step - loss: 2.2435
Epoch 117/500
5/5 [==============================] - 8s 2s/step - loss: 2.2233
Epoch 118/500
5/5 [==============================] - 8s 2s/step - 

5/5 [==============================] - 8s 2s/step - loss: 0.7985
Epoch 210/500
5/5 [==============================] - 8s 2s/step - loss: 0.7942
Epoch 211/500
5/5 [==============================] - 8s 2s/step - loss: 0.7865
Epoch 212/500
5/5 [==============================] - 8s 2s/step - loss: 0.7754
Epoch 213/500
5/5 [==============================] - 8s 2s/step - loss: 0.7673
Epoch 214/500
5/5 [==============================] - 8s 2s/step - loss: 0.7580
Epoch 215/500
5/5 [==============================] - 8s 2s/step - loss: 0.7562
Epoch 216/500
5/5 [==============================] - 8s 2s/step - loss: 0.7459
Epoch 217/500
5/5 [==============================] - 8s 1s/step - loss: 0.7343
Epoch 218/500
5/5 [==============================] - 8s 2s/step - loss: 0.7266
Epoch 219/500
5/5 [==============================] - 8s 2s/step - loss: 0.7238
Epoch 220/500
5/5 [==============================] - 8s 1s/step - loss: 0.7138
Epoch 221/500
5/5 [==============================] - 8s 2s/step - 

5/5 [==============================] - 3s 551ms/step - loss: 0.3211
Epoch 313/500
5/5 [==============================] - 3s 565ms/step - loss: 0.3131
Epoch 314/500
5/5 [==============================] - 3s 573ms/step - loss: 0.3120
Epoch 315/500
5/5 [==============================] - 3s 646ms/step - loss: 0.3116
Epoch 316/500
5/5 [==============================] - 3s 613ms/step - loss: 0.3121
Epoch 317/500
5/5 [==============================] - 3s 607ms/step - loss: 0.3080
Epoch 318/500
5/5 [==============================] - 3s 614ms/step - loss: 0.3052
Epoch 319/500
5/5 [==============================] - 3s 531ms/step - loss: 0.3061
Epoch 320/500
5/5 [==============================] - 3s 532ms/step - loss: 0.2983
Epoch 321/500
5/5 [==============================] - 3s 530ms/step - loss: 0.3026
Epoch 322/500
5/5 [==============================] - 3s 535ms/step - loss: 0.3003
Epoch 323/500
5/5 [==============================] - 3s 531ms/step - loss: 0.2960
Epoch 324/500
5/5 [===========

5/5 [==============================] - 3s 591ms/step - loss: 0.1713
Epoch 413/500
5/5 [==============================] - 3s 560ms/step - loss: 0.1712
Epoch 414/500
5/5 [==============================] - 3s 651ms/step - loss: 0.1677
Epoch 415/500
5/5 [==============================] - 3s 533ms/step - loss: 0.1704
Epoch 416/500
5/5 [==============================] - 3s 666ms/step - loss: 0.1694
Epoch 417/500
5/5 [==============================] - 3s 590ms/step - loss: 0.1664
Epoch 418/500
5/5 [==============================] - 3s 644ms/step - loss: 0.1657
Epoch 419/500
5/5 [==============================] - 3s 570ms/step - loss: 0.1665
Epoch 420/500
5/5 [==============================] - 3s 562ms/step - loss: 0.1656
Epoch 421/500
5/5 [==============================] - 3s 621ms/step - loss: 0.1660
Epoch 422/500
5/5 [==============================] - 3s 585ms/step - loss: 0.1627
Epoch 423/500
5/5 [==============================] - 3s 591ms/step - loss: 0.1628
Epoch 424/500
5/5 [===========

In [20]:
model.save('sch.h5')

Inference models

In [21]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=(256,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(256,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [22]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( input_word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_input_length , padding='post')

In [25]:
enc_model , dec_model = make_inference_models()
for epoch in range( encoder_input_data.shape[0] ):
    states_values = enc_model.predict( str_to_tokens( input( 'User: ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = output_worddic['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in output_worddic.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > max_output:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( "Bot:" +decoded_translation.replace(' end', '') )
    print()

User: HI
1/1 [==============================] - 0s 17ms/step
Bot: hello

User: what is your name
1/1 [==============================] - 0s 16ms/step
Bot: computers is a human except that we lack the sun

User: what are your interest
1/1 [==============================] - 0s 17ms/step
Bot: i like to me on the internet

User: hello
1/1 [==============================] - 0s 16ms/step
Bot: hi

User: end


KeyError: 'end'